In [1]:
import glob
import os
from IPython.display import display, Markdown
from tqdm import tqdm
from ipywidgets import FloatSlider, IntSlider
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from skimage.filters import threshold_otsu
from skimage import measure
from skimage import morphology
from skimage.color import label2rgb
from skimage.measure import regionprops
from skimage.draw import circle
from skimage.filters import gaussian
from skimage.segmentation import clear_border

In [3]:
PATH = "carta-laboral-images/"
images = (glob.glob(PATH + "*.png") + glob.glob(PATH + "*.jpg") + glob.glob(PATH + "*.jpeg"))
images.sort()

In [4]:
def get_a4_constant(blobs_labels, min_area):

    total_area = 0
    counter = 0
    for region in regionprops(blobs_labels):

        if region.area > min_area:
            total_area = total_area + region.area
            counter = counter + 1

    average = (total_area / counter)
    a4_constant = ((average / 84.0) * 250.0) + 100
    
    return a4_constant

In [5]:
@interact
def segment_image(image = images,
                  th1=200,
                  min_area=300
                  ):
    
    img = cv2.imread(f'{image}', 0)
    
    original = cv2.imread(f'{image}')

    img = cv2.threshold(img, th1, 255, cv2.THRESH_BINARY)[1]

    open_img = morphology.opening(img, morphology.square(3))

    gaussian_img = gaussian(open_img)

    blobs = gaussian_img > gaussian_img.mean()

    blobs_labels = measure.label(blobs, background=1)

    a4_constant = get_a4_constant(blobs_labels, min_area)

    b = morphology.remove_small_objects(blobs_labels, a4_constant)
    
    image_label_overlay = label2rgb(b, image=original, bg_label=0)
    
    plt.figure(figsize=(40,20))
    
    plt.imshow(image_label_overlay)

    plt.show()

interactive(children=(Dropdown(description='image', options=('carta-laboral-images/1-0.png', 'carta-laboral-im…